### 1.0 Import modules

In [35]:
import pandas as pd
import numpy as np
from datetime import datetime
from rapidfuzz import process, utils as fuzz_utils
GOOGLE_SHEET_MASTER_LIST_URL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSWbwrsqF-c---4lfw0LZWymd-f8sy8sLYkXgzh0OyeGATWwrvv7V1Mq5BcApn7F_-WYKP1KXy5shKw/pub?gid=240503925&single=true&output=csv'


### 1.1 Load the data into data frames


In [44]:
master_list_schools_df = pd.read_csv(GOOGLE_SHEET_MASTER_LIST_URL,usecols={'id','california_oid','phone','original_name_x','participating','rowid','school','district_y','address_y'})
# master_list_schools_df.columns# master_list_schools_df = 
master_list_schools_df.loc[master_list_schools_df['school'].str.len() < 3, 'school'] = master_list_schools_df['original_name_x']
master_list_schools_df

cali_id_df = master_list_schools_df.loc[master_list_schools_df['california_oid'].notnull()]
participating_df = master_list_schools_df.loc[master_list_schools_df['participating'] == 1]
# participating_df.groupby('california_oid').count()
participating_df

,id,california_oid,phone,original_name_x,participating,rowid,school,district_y,address_y
0,0,1.964352e+13,(310) 263-4400,Hawthorne High,True,306.0,Hawthorne High (Centinela Valley Union High),Centinela Valley Union High,4859 West El Segundo Blvd.
1,1,1.964352e+13,(310) 263-3100,Lawndale High,True,307.0,Lawndale High (Centinela Valley Union High),Centinela Valley Union High,14901 South Inglewood Ave.
2,2,1.964352e+11,(310) 263-2200,Leuzinger High,True,308.0,Leuzinger High (Centinela Valley Union High),Centinela Valley Union High,4118 West Rosecrans Ave.
3,3,1.964352e+11,(310) 263-3264,R. K. Lloyde High,True,310.0,R. K. Lloyde High (Centinela Valley Union High),Centinela Valley Union High,4951 Marine Ave.
4,4,NaN,(310) 221-0430,Alliance Alice M. Baxter College-Ready High Sc...,True,0.0,Alliance Alice M. Baxter College-Ready High Sc...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1480,1480,1.964882e+13,(626) 396-5840,Washington Elementary,True,221.0,Washington Elementary (Bellflower Unified),Bellflower Unified,9725 East Jefferson St.
1481,1481,1.964881e+11,(626) 396-5690,Willard Elementary,True,1143.0,Willard Elementary (Long Beach Unified),Long Beach Unified,1055 Freeman Ave.
1482,1482,1.964734e+13,(323) 441-3113,Cathedral High School,True,862.0,Cathedral High School (Independent Charter Sch...,Independent Charter Schools District,1253 Bishops Rd.
1483,1483,1.964734e+13,(323) 284-7266,Episcopal School of Los Angeles,True,825.0,The Episcopal School of Los Angeles (Independe...,Independent Charter Schools District,6325 Santa Monica Blvd.


In [65]:
california_schools_data = "../data/CDESchoolDirectoryExportAugust2022.csv"

california_schools = pd.read_csv(california_schools_data,
    usecols={'CDS Code','County','Status','District','School',"Latitude","Longitude","Last Update",'Street Address','Street City',"Phone"},encoding='latin')
california_schools.columns = ["oid","county","district","name","status","latitude","longitude","last_update","address","city","default_phone"]

california_schools = california_schools.fillna('')
la_schools_df = california_schools[california_schools["county"] == "Los Angeles"]

la_schools_df['original_name'] = la_schools_df['name']
name = la_schools_df['name'].str.lower().str.replace("[[:punct:]]","-",regex=True).replace({'\([a-zA-Z\s\.\-\/]*\(*[a-zA-Z\s\.\-\/]*\)*[a-zA-Z\s\.\-\/]*\)$'},'',regex=True)

la_schools_df.loc[la_schools_df['district'] == "Los Angeles Unified", 'district'] = "LAUSD"
simple_district = la_schools_df['district'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
simple_address = la_schools_df['address'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')

la_schools_df['simple_name'] = simple_address+"-"+name+"-"+simple_district
la_schools_df['simple_name'].fillna('none',inplace=True)

# la_schools_df['oid_key'] = la_schools_df['oid'].astype(str).str.replace(".0","")
la_schools_df = la_schools_df.loc[la_schools_df['status'] != "Closed"]
la_schools_df.drop_duplicates(['oid'])
la_schools_df
# master_list_subset_with_oid['oid_key'] = master_list_subset_with_oid['california_oid'].astype(str).str.replace(".0","")


<ipython-input-65-1f1f5405c466>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_schools_df['original_name'] = la_schools_df['name']
c:\Users\hikou\.conda\envs\metro\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-65-1f1f5405c466>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as

,oid,county,district,name,status,latitude,longitude,last_update,address,city,default_phone,original_name,simple_name,oid_key
0,19642121930346,Los Angeles,ABC Unified,ABC Adult,Active,33.878924,-118.071286,7/19/2021,12254 Cuesta Dr.,Cerritos,(562) 229-7960,ABC Adult,12254-cuesta-dr-abc adult-abc-unified,196421219346
2,19642121995596,Los Angeles,ABC Unified,ABC Secondary (Alternative),Active,33.881547,-118.046358,11/5/2021,16534 South Carmenita Rd.,Cerritos,(562) 229-7768,ABC Secondary (Alternative),16534-south-carmenita-rd-abc secondary -abc-un...,19642121995596
4,19642126010862,Los Angeles,ABC Unified,Aloha Elementary,Active,33.835176,-118.083725,11/5/2021,11737 East 214th St.,Lakewood,(562) 229-7825,Aloha Elementary,11737-east-214th-st-aloha elementary-abc-unified,1964212862
5,19642121930361,Los Angeles,ABC Unified,Artesia High,Active,33.845962,-118.074404,11/5/2021,12108 East Del Amo Blvd.,Lakewood,(562) 229-7700,Artesia High,12108-east-del-amo-blvd-artesia high-abc-unified,196421219361
7,19642126071369,Los Angeles,ABC Unified,Bragg Elementary,Active,33.855106,-118.089062,11/5/2021,11501 Bos St.,Cerritos,(562) 229-7830,Bragg Elementary,11501-bos-st-bragg elementary-abc-unified,196421271369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5822,19768690119016,Los Angeles,Wiseburn Unified,Da Vinci Science,Active,33.917908,-118.384081,11/5/2021,201 N. Douglas St.,El Segundo,(310) 725-5800,Da Vinci Science,201-n-douglas-st-da vinci science-wiseburn-uni...,1976861116
5823,19768696023824,Los Angeles,Wiseburn Unified,Juan Cabrillo Elementary,Active,33.909520,-118.372738,11/5/2021,5309 West 135th St.,Hawthorne,(310) 725-5400,Juan Cabrillo Elementary,5309-west-135th-st-juan cabrillo elementary-wi...,197686923824
5824,19768696023808,Los Angeles,Wiseburn Unified,Juan De Anza Elementary,Active,33.922918,-118.372077,11/5/2021,12110 Hindry Ave.,Hawthorne,(310) 725-2100,Juan De Anza Elementary,12110-hindry-ave-juan de anza elementary-wiseb...,1976869238
5826,19768696023832,Los Angeles,Wiseburn Unified,Richard Henry Dana Middle,Active,33.909404,-118.376654,11/5/2021,5504 West 135th St.,Hawthorne,(310) 725-4700,Richard Henry Dana Middle,5504-west-135th-st-richard henry dana middle-w...,197686923832


### 2.0 Subset and join data with California IDs


In [64]:
# master_list_subset_no_oid
# from argparse import ONE_OR_MORE


master_list_subset_with_oid = master_list_schools_df[master_list_schools_df['california_oid'].notnull()]
master_list_subset_with_oid
master_list_subset_with_oid['oid_key'] = master_list_subset_with_oid['california_oid'].astype(str).str.replace(".0","")
master_list_subset_with_oid 
#  """1141
la_schools_df.drop_duplicates(['oid_key'], keep='first')
inner_join_master_list_with_schools = pd.merge(master_list_subset_with_oid, la_schools_df, left_on='oid_key', right_on='oid_key', how='left')
inner_join_master_list_with_schools

<ipython-input-64-3a6f8b349686>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  master_list_subset_with_oid['oid_key'] = master_list_subset_with_oid['california_oid'].astype(str).str.replace(".0","")
<ipython-input-64-3a6f8b349686>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_list_subset_with_oid['oid_key'] = master_list_subset_with_oid['california_oid'].astype(str).str.replace(".0","")


KeyError: 'oid'

In [ ]:
# inner_join_master_list_with_schools.loc[inner_join_master_list_with_schools['phone'].isnull(), 'participating'] 
inner_join_master_list_with_schools.loc[inner_join_master_list_with_schools['phone'].isnull()]
# null_phone_df  = inner_join_master_list_with_schools[inner_join_master_list_with_schools['phone'].str.len() < 15]

inner_join_master_list_with_schools.loc[inner_join_master_list_with_schools['phone'].isnull(), 'phone'] = inner_join_master_list_with_schools['default_phone']
# null_phone_df  = inner_join_master_list_with_schools[inner_join_master_list_with_schools['phone'].str.contains("No Data")]
# null_phone_df
# inner_join_master_list_with_schools.columns
inner_join_master_list_with_schools_df_to_combine = inner_join_master_list_with_schools[['california_oid','phone','participating','rowid','school','district_y','address_y','status','latitude','longitude']]
inner_join_master_list_with_schools.drop_duplicates(['school','address_y'], keep='first', inplace=True)
inner_join_master_list_with_schools_df_to_combine


### 3.0 Subset the data with no California IDs


In [ ]:
# master_list_subset_no_oid = master_list_schools_df[master_list_schools_df['california_oid'].isnull()]
name = master_list_schools_df['school'].str.replace("[[:punct:]]","-",regex=True).replace({'\([a-zA-Z\s\.\-\/]*\(*[a-zA-Z\s\.\-\/]*\)*[a-zA-Z\s\.\-\/]*\)$'},'',regex=True)

simple_district = master_list_schools_df['district_y'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
simple_address = master_list_schools_df['address_y'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
simple_name = name.str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','').str.split().str.get(0)
key = simple_address+"-"+simple_name
clean_key = key.str.replace("---","-")
master_list_schools_df['tap_key'] = clean_key
master_list_schools_df['tap_key'].fillna('none',inplace=True)
# master_list_subset_no_oid.drop_duplicates(subset=['tap_key'], keep='first', inplace=True)
# master_list_subset_no_oid
final_df = master_list_schools_df.loc[master_list_schools_df['participating'] == 1]
final_df


### 3.1 Merge the data with no IDs using `rapid fuzzy` 

In [ ]:

def fuzzy_merge(baseFrame, compareFrame, baseKey, compareKey, threshold=86, limit=1, how='left'):
    s_mapping = {x: fuzz_utils.default_process(x) for x in compareFrame[compareKey]}

    m1 = baseFrame[baseKey].apply(lambda x: process.extract(
      fuzz_utils.default_process(x), s_mapping, limit=limit, score_cutoff=threshold, processor=None
    ))
    baseFrame['Match'] = m1

    m2 = baseFrame['Match'].apply(lambda x: ', '.join(i[2] for i in x))
    baseFrame[baseKey] = m2.replace("",np.nan)

    return baseFrame.merge(compareFrame, left_on=baseKey, right_on=compareKey, how=how)

merged_df = fuzzy_merge(master_list_schools_df, la_schools_df, 'tap_key', 'simple_name',how='left')
# merged_df = fuzzy_merge(la_schools_df, go_pass_schools 'name', 'simple_name',how='right')
merged_df

### 3.2 Clean up the results


In [ ]:
# merged_df.columns
# merged_df_to_combine = merged_df[['id','california_oid','phone','participating','rowid','school','district_y','address_y','latitude','longitude','email']]
merged_df.loc[merged_df['phone'].isnull(), 'phone'] = merged_df['default_phone']
merged_df_to_combine = merged_df[['phone','participating','rowid','school','district_y','address_y','Match','oid','status','latitude','longitude']]
# merged_df_to_combine.drop_duplicates(subset=['tap_key'], keep='first', inplace=True)
merged_df_to_combine.rename(columns = {'oid':'california_oid'}, inplace = True)
merged_df_to_combine


# merged_df_to_combine

### 4.0 Combining final data frame

In [ ]:
combined_df = pd.concat([inner_join_master_list_with_schools_df_to_combine, merged_df_to_combine], ignore_index=True)
combined_df.reset_index(drop=True, inplace=True)
combined_df.index.names = ['id']
combined_df['california_oid'] = combined_df['california_oid'].astype(str).str.replace(".0","")
combined_df['rowid'] = combined_df['rowid'].astype(str).str.replace(".0","")
final_df = combined_df.loc[combined_df['participating'] == 1]
# final_df.drop_duplicates(['school','address_y'], keep='first', inplace=True)
final_df


### 5.0 Writing JSON and CSV Outputs

In [ ]:
output_file_name = "../data/final_tap_data_post_processing_master_list.csv"
combined_df.to_csv(output_file_name)

output_json = "../src/data/schools.json"
json_file = combined_df.to_json(orient='records',index=True) 
with open(output_json, 'w') as f:
    f.write(json_file)